***
<center><img src='https://github.com/HesamAsad/HesamAsad.github.io/blob/main/images/LOGO_EN_blue.png?raw=true' width=250/></center>

# <center>
## <center><span style='font-family:"Times New Roman"'><span style='background:dark'>  <font color='#1966ab'>$\LARGE{\text{Sharif University of Technology}}$</font>  <br>  <br>  <font color='#FFFFCC'>$\Large{\text{Masoud Tahmasbi}}$ </font> <br> <font color='#FFFFCC'>$\normalsize {\text{Student ID: } 402200275}$ </font>  <br>  <br> <font color='#33FFCC'>$\large{\text{EE577: Massive Data Analytics}}$</font> <br>  <font color='#FFFF00'>$\normalsize{\text{Assignment #3 - Twitter User Engagement Recommender System}}$ </font>
<br>

***

# $\text{Data Preparation}$

In [ ]:
! pip install pyspark

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

!cp '/content/gdrive/My Drive/twitter_data_v2.tar.gz' twitter_data
!tar -xzvf "/content/twitter_data"

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("HW3") \
    .master("local[*]") \
    .getOrCreate()

sc=spark.sparkContext

In [ ]:
twitter_rdd = sc.textFile("twitter_data_v2.jsonl")

In [ ]:
import json
parsed_rdd = twitter_rdd.map(lambda x: json.loads(x))
parsed_rdd.count()

4175860

In [ ]:
parsed_rdd.takeSample(False,1,112)

[{'in_reply_to_status_id_str': None,
  'in_reply_to_user_id_str': None,
  'in_reply_to_status_id': None,
  'in_reply_to_user_id': None,
  'in_reply_to_screen_name': None,
  'truncated': False,
  'is_quote_status': False,
  'retweet_count': 20,
  'reply_count': 0,
  'quote_count': 0,
  'favorite_count': 0,
  'favorited': False,
  'retweeted': True,
  'possibly_sensitive': None,
  'lang': 'fa',
  'geo': None,
  'view_count': 0,
  'tweet_type': 'retweeted',
  'emojis': None,
  'text': 'فک کن بچه رو کشتن، بچه ۲۰ ساله، حالام میخوان مادرش رو ۵ سال ببرن زندان.\nواقعا آخوند چه کثافتی هست که این بی پدرا انداختن تو زندگی ما.\n#محمدجواد_زاهدی\n#مهسا_یزدانی',
  'user': {'id': '1565039849287172096',
   'id_str': '1565039849287172096',
   'created_at': 1661969566,
   'name': 'Negin',
   'screen_name': 'Negin88419550',
   'location': '',
   'description': '',
   'verified': False,
   'followers_count': 20,
   'friends_count': 4,
   'listed_count': 0,
   'favourites_count': 15093,
   'statuses_count':

In [ ]:
# parsed_rdd.takeSample(False,1,12)

In [ ]:
# parsed_rdd.takeSample(False,1,312)

In [ ]:
# parsed_rdd.takeSample(False,1,98)

In [ ]:
# parsed_rdd.takeSample(False,1,48)

***
# $\text{Creating User Engagement RDD}$

In [ ]:
def scoring(tweet, generate_score=3, quote_score=2, reply_score=2, retweet_score=1):
    if tweet.get('tweet_type') == 'generated':
        try:
            generator_uid = tweet.get('user').get('id')
        except: return []
        try:
            generated_tweet_id = tweet.get('id')
        except: return []

        return [(generator_uid,
                 [(generator_uid,generated_tweet_id,generate_score)])]
        # structure: (generator_uid, [(generator_uid,generated_tweet_id,generate_score)])

    if tweet.get('tweet_type') == 'quoted':
        try:
            quoting_uid = tweet.get('user').get('id')
        except: return []
        try:
            quoted_uid = tweet.get('quoted_status').get('user').get('id')
        except: return []
        try:
            quoted_tweet_id = tweet.get('quoted_status').get('id')
        except: return []

        return [(quoting_uid,
                 [(quoted_uid,quoted_tweet_id,quote_score)])]
    # structure: (quoting_uid, [(quoted_uid,quoted_tweet_id,quote_score)])

    if tweet.get('tweet_type') == 'replied':
        try:
            replying_uid = tweet.get('user').get('id')
        except: return []
        try:
            replied_uid = tweet.get('in_reply_to_user_id_str')
        except: return []
        try:
            replied_tweet_id = tweet.get('in_reply_to_status_id_str')
        except: return []

        return [(replying_uid,
                 [(replied_uid,replied_tweet_id,reply_score)])]
    # structure: (replying_uid, [(replied_uid,replied_tweet_id,reply_score)])

    if tweet.get('tweet_type') == 'retweeted':
        try:
            retweeting_uid = tweet.get('user').get('id')
        except: return []
        try:
            retweeted_uid = tweet.get('retweeted_status').get('user').get('id')
        except: return []
        try:
            retweeted_tweet_id = tweet.get('retweeted_status').get('id')
        except: return []

        return [(retweeting_uid,
                 [(retweeted_uid,retweeted_tweet_id,retweet_score)])]
    # structure: (retweeting_uid, [(retweeted_uid,retweeted_tweet_id,retweet_score)])

In [ ]:
scores_rdd = parsed_rdd.flatMap(lambda x:scoring(x))
scores_rdd.count()

3990340

In [ ]:
grouped_rdd = scores_rdd.reduceByKey(lambda a,b: a+b)
grouped_rdd.count()

296851

***
# $\text{Type #1 User}$

In [ ]:
target_user = grouped_rdd.takeSample(False,1,55)[0]
target_user

('1697206791715635200',
 [('1710302619371982848', '1736250426230493213', 1),
  ('1710302619371982848', '1736250178363883872', 1),
  ('1710302619371982848', '1736249928018764235', 1),
  ('1578454116107517962', '1735951927584682444', 2),
  ('1633911157755879425', '1736108738921033809', 1),
  ('1654409094076268546', '1736287140860829901', 1),
  ('1205723159984574464', '1736622445505741159', 2),
  ('1654179648761102337', '1734655981412573264', 1),
  ('1654179648761102337', '1734657820010991962', 1),
  ('1654179648761102337', '1734661707908120774', 1),
  ('1654179648761102337', '1734661220601352349', 1),
  ('1654179648761102337', '1734659777710498122', 1),
  ('1654179648761102337', '1734658201302577473', 1),
  ('1398658879827886085', '1734656185226424457', 2),
  ('1710302619371982848', '1734799518305034308', 2),
  ('1710302619371982848', '1734798971577536976', 1),
  ('1710302619371982848', '1734798971577536976', 2),
  ('1710302619371982848', '1734798173476950425', 1),
  ('169720679171563520

In [ ]:
def target_user_tweets_parser(target_user,tweet):
    target_user_tweets_id = [tweets[1] for tweets in target_user[1]]

    if tweet.get('id') in target_user_tweets_id:
        return [(tweet.get('text'),tweet.get('tweet_type'))]
    else:
        return []

In [ ]:
target_user_tweets = parsed_rdd.flatMap(lambda x: target_user_tweets_parser(target_user,x))
target_user_tweets.collect()

[('وصیت نامه یکی از  شهدای بزرگوار :  خدایا نشود که در زندگی دو دستی به درخت زندگی بچسبم و همچون میوه ایی پوسیده بیافتم خدایا تا ایمانی هست مرا برای خود قربانی کن\n#گمنام_مثل_مادر\n#لشکر_علویان https://t.co/CzBw8KoKmM',
  'generated'),
 ('زندگی پر از انتخاب هایی است که مسیر ها رو عوض می\u200cکنند، اینکه قهرمان زندگی چه شخصی باشه در عاقبت بخیری خیلی مهمه\n#گمنام_مثل_مادر\n#لشکر_علویان https://t.co/QGEkchpZl4',
  'generated'),
 ('حضور مردم قدر شناس دیار علویان در مراسم وداع با لاله\u200cها\n#وداع_با_لاله\u200cها\n#گمنام_مثل_مادر https://t.co/JWvgiokZzn',
  'generated'),
 ('دیار علویان و مردم همیشه در صحنه\n#وداع_با_لاله\u200cها\n#گمنام_مثل_مادر https://t.co/1NNoFp1AGe',
  'generated'),
 ('در پی اسم و...\n#گمنام_مثل_مادر\n#وداع_با_لاله\u200cها https://t.co/QgF5gIXki9',
  'generated'),
 ('قبلا باور نمی کردم؛ولی باچشمان خودم می بینم همونایکه دم از وحدت و برادری می زنند،متأسفانه اینجا علیه مقدسات اهل\u200cسنت قلم\u200c می زنند،توهین می کنند و یه آب هم میزنن تنگش!نمی دونم فازشون چیه!شمایکه خو

In [ ]:
import numpy as np

In [ ]:
def pearson_coeff(target_user,user_x):

    target_user_tweets_ids = [tweets[1] for tweets in target_user[1]]
    user_x_tweets_ids = [tweets[1] for tweets in user_x[1]]
    S_xy =  list(set(target_user_tweets_ids) & set(user_x_tweets_ids))
    r_x = np.array(list(set(tweets[2] for tweets in target_user[1]
                    if tweets[1] in S_xy)))

    r_y = np.array(list(set(tweets[2] for tweets in user_x[1]
                    if tweets[1] in S_xy)))

    if r_x.size == 0:
        return [(0,user_x[0])]

    # ignore removing the mean
    sim_xy = (sum( (r_x)*(r_y) ) )\
            /((np.sqrt(sum( (r_x)**2) ))*(np.sqrt(sum( (r_y)**2) )))

    return [(sim_xy,user_x[0])]

In [ ]:
similars_rdd = grouped_rdd.flatMap(lambda x:pearson_coeff(target_user,x))
similars_rdd.count()

296851

In [ ]:
sorted_similars_rdd = similars_rdd.sortByKey()
sorted_similars_rdd.count()

296851

In [ ]:
similar_users = sorted_similars_rdd.top(20)
similar_users

[(1.6035674514745464, '1724483603600486400'),
 (1.6035674514745464, '1632351425383522312'),
 (1.6035674514745464, '1487097029990002688'),
 (1.3416407864998738, '774470366676877312'),
 (1.3416407864998738, '1725652340043100160'),
 (1.3416407864998738, '1725280261300965378'),
 (1.3416407864998738, '1721135891308007424'),
 (1.3416407864998738, '1719916251404021760'),
 (1.3416407864998738, '1719773177503682560'),
 (1.3416407864998738, '1719696919692292096'),
 (1.3416407864998738, '1718837061237530624'),
 (1.3416407864998738, '1710302619371982848'),
 (1.3416407864998738, '1708866964020514816'),
 (1.3416407864998738, '1704606183858880513'),
 (1.3416407864998738, '1704531218174251012'),
 (1.3416407864998738, '1692155672807325699'),
 (1.3416407864998738, '1689309272180482048'),
 (1.3416407864998738, '1685928050708418560'),
 (1.3416407864998738, '1675154446966312962'),
 (1.3416407864998738, '1648401324608827392')]

In [ ]:
def similar_tweets_score(similar_users,user_tweets):
    similar_users_ids = [x[1] for x in similar_users]
    similar_users_similarity = [x[0] for x in similar_users]

    if user_tweets[0] in similar_users_ids:
        tweets_ids = [x[1] for x in user_tweets[1]]
        similarity = similar_users_similarity[similar_users_ids.index(user_tweets[0])]
        tweets_scores = list(np.array([x[2] for x in user_tweets[1]])*similarity)
        score_id = list(zip(tweets_scores,tweets_ids))

        return score_id
    else:
        return []

In [ ]:
recomm_tweets_ids = grouped_rdd.flatMap(lambda x: similar_tweets_score(similar_users,x)).collect()

In [ ]:
recomm_tweets_ids.sort(key=lambda a: a[0],reverse = True)

In [ ]:
recommend_tweets_ids = recomm_tweets_ids[0:20]
recommend_tweets_ids

[(4.810702354423639, '1734904359844982845'),
 (4.810702354423639, '1734599807044349982'),
 (4.810702354423639, '1734204787200004101'),
 (4.024922359499621, '1735389039786668038'),
 (4.024922359499621, '1734604136702935127'),
 (4.024922359499621, '1735901407872475393'),
 (4.024922359499621, '1735901649871253757'),
 (4.024922359499621, '1736285402858688631'),
 (4.024922359499621, '1736285577358573735'),
 (4.024922359499621, '1736348680079233107'),
 (4.024922359499621, '1736349049442210119'),
 (4.024922359499621, '1736349214286688597'),
 (4.024922359499621, '1736349372965655032'),
 (4.024922359499621, '1734182827418095904'),
 (4.024922359499621, '1736000149338325236'),
 (4.024922359499621, '1734227232418075035'),
 (4.024922359499621, '1734234103837327546'),
 (4.024922359499621, '1735690223877501163'),
 (4.024922359499621, '1734273745986994506'),
 (4.024922359499621, '1735714482779750596')]

In [ ]:
def recommended_tweets_parser(recommend_tweets_ids,tweet):
    ids = [a[1] for a in recommend_tweets_ids]
    if tweet.get('id') in ids:
        return [(tweet.get('text'),)]
    else:
        return []

In [ ]:
recommended_tweets = parsed_rdd.flatMap(lambda x: recommended_tweets_parser(recommend_tweets_ids,x))
recommended_tweets.collect()

[('خنده\u200cهاشان خاکی بود\nگریه\u200c هاشان آسمانی ؛  \nبی\u200c ریا و خاکی که باشی\nآسمانی\u200cها خاطرخواهت می\u200cشوند \n#وداع_با_لاله\u200cها \n#روایت_عشق',),
 ('چه سروهایی که قطعه قطعه شدند\nعلی اکبر هایی که رفتند \nو علی اصغر برگشتند...\n\n#وداع_با_لاله\u200cها \n#روایت_عشق https://t.co/Wtpvs9DrN2',),
 ('یک چفیه پر از بهار، باور کردم\n\nموسیقی انتظار، باور کردم\n\nمن عاشقی تو را به روی مین\u200cها\n\nدر لحظه\u200cی انفجار باور کردم\n#وداع_با_لاله\u200cها\n#روایت_عشق https://t.co/KhlAqtjsut',),
 ('ای دوست به شهر ما صفا آوردی\n\nاین رایحه را تو از کجا آوردی؟\n\nانگار نماز عشق را سر سرخ\n\nبر تربت لاله\u200cها به جا آوردی\n\n#وداع_با_لاله\u200cها\n#روایت_عشق https://t.co/29dC9xkqxC',),
 ('🌹قرار آسمانی با ستاره آسمان خوشنامی در #هادیشهر\n#گمنام_مثل_مادر \n#لشکر_علویان https://t.co/885JANG6B5',),
 ('🇮🇷گل اشکم شبی وا می شد ای کاش\nهمه دردم مداوا می شد ای کاش\nبه هرکس قسمتی دادی خدایا\nشهادت قسمت ما می شد ای کاش\n#گمنام_مثل_مادر\n#لشکر_علویان',),
 ('🏴سلام بر شهیدان گمنام \n\nگمنام در

***
# $\text{Type #2 User}$

In [ ]:
target_user = grouped_rdd.takeSample(False,1,1000)[0]
target_user

('1036885525326647296',
 [('1591547142136627202', '1736619970027462949', 1),
  ('1227221292685938688', '1734241138918175049', 1),
  ('1227221292685938688', '1733918649746919740', 1),
  ('1591547142136627202', '1734596007038517749', 1),
  ('1591547142136627202', '1735659470351433916', 1),
  ('1591547142136627202', '1735310756118155394', 1),
  ('1591547142136627202', '1736442052454310094', 1),
  ('1591547142136627202', '1736076770128851061', 1),
  ('1591547142136627202', '1736096711972475174', 1)])

In [ ]:
target_user_tweets = parsed_rdd.flatMap(lambda x: target_user_tweets_parser(target_user,x))
target_user_tweets.collect()

[('در مملکتی که شورای نگهبان مصلحت اندیشی کند و مجمع تشخیصِ مصلحت کار فقهی!هیچ چیز درست نخواهد شد .\n#روحانی',
  'generated'),
 ('#اینبار_فرق_میکند\nلطفا بفرمایید از بنده انتظار دارید در انتخابات پیش رو چه کنشگری داشته باشم \nصد البته نظر شما برای حقیر مهم و تعیین کننده خواهد بود\nفی المثل آیا با گروهای دیگر اجماع کنیم یا خیر ؟\nاگر بله چه اشخاص ، گروه ها و جریاناتی؟\nخطوط قرمز شما چیست ؟\nچه مسایلی برایتان اولویت دارد ؟\nو ….',
  'generated'),
 ('یک بار  برای همیشه باید تکلیفمان را با بی تفاوتی و در مواردی شیطنت دولت پاکستان در برقراری امنیت مرزها مشخص کنیم\nباید تکلیفمان را با #سربازی_اجباری مشخص کنیم\nباید با کمبود تجهیزات و نبود آموزش نیروهای مرزبانی مشخص کنیم \nباید با عقبه ایدئولوژیک تروریست ها در مرزهای داخلی ! مشخص کنیم.\nباید تکلیفمان را با مسئولین بی عرضه ای که در تهران نشسته و برای مرزها نسخه غلط می پیچند مشخص کنیم \n#راسک\n#ایران_تسلیت',
  'generated'),
 ('@M_Afsari2 @RezaGholami1354 در باب گفتگو هم قرار بود دیروز ایشان تشریف بیاورند دفتر ما که ظاهرا کاری برایشان پیش آمد و 

In [ ]:
similars_rdd = grouped_rdd.flatMap(lambda x:pearson_coeff(target_user,x))
similars_rdd.count()

296851

In [ ]:
sorted_similars_rdd = similars_rdd.sortByKey()
sorted_similars_rdd.count()

296851

In [ ]:
similar_users = sorted_similars_rdd.top(20)
similar_users

[(1.3416407864998738, '988684923203710976'),
 (1.3416407864998738, '952463173868818432'),
 (1.3416407864998738, '951201559597191169'),
 (1.3416407864998738, '948126072603754496'),
 (1.3416407864998738, '941220112845164544'),
 (1.3416407864998738, '919323412195102721'),
 (1.3416407864998738, '918806712995348480'),
 (1.3416407864998738, '911453224049164288'),
 (1.3416407864998738, '901777753833332736'),
 (1.3416407864998738, '880353064477233152'),
 (1.3416407864998738, '878382611730751488'),
 (1.3416407864998738, '878217414323961856'),
 (1.3416407864998738, '874242781790818305'),
 (1.3416407864998738, '855438360730578946'),
 (1.3416407864998738, '834665971075907585'),
 (1.3416407864998738, '822564412997402624'),
 (1.3416407864998738, '819266108351856640'),
 (1.3416407864998738, '818514349782405120'),
 (1.3416407864998738, '818336090998374400'),
 (1.3416407864998738, '814591408862416897')]

In [ ]:
recomm_tweets_ids = grouped_rdd.flatMap(lambda x: similar_tweets_score(similar_users,x)).collect()

In [ ]:
recomm_tweets_ids.sort(key=lambda a: a[0],reverse = True)

In [ ]:
recommend_tweets_ids = recomm_tweets_ids[0:20]
recommend_tweets_ids

[(4.024922359499621, '1736200632980103386'),
 (4.024922359499621, '1736200676332417325'),
 (4.024922359499621, '1736571240553283866'),
 (4.024922359499621, '1736571274892296333'),
 (4.024922359499621, '1735849256391475619'),
 (4.024922359499621, '1735849284082303219'),
 (4.024922359499621, '1735867134490607746'),
 (4.024922359499621, '1736640572977250357'),
 (4.024922359499621, '1736663226119512083'),
 (4.024922359499621, '1735958921079112138'),
 (4.024922359499621, '1736695367943561440'),
 (4.024922359499621, '1736703321509892379'),
 (4.024922359499621, '1736722218724274318'),
 (4.024922359499621, '1735994535237325134'),
 (4.024922359499621, '1736358408054296791'),
 (4.024922359499621, '1735647971692204432'),
 (4.024922359499621, '1735648652553560540'),
 (4.024922359499621, '1736760514624016488'),
 (4.024922359499621, '1735654953446736373'),
 (4.024922359499621, '1735672954925515228')]

In [ ]:
recommended_tweets = parsed_rdd.flatMap(lambda x: recommended_tweets_parser(recommend_tweets_ids,x))
recommended_tweets.collect()

[('اَمَّن یُّجیبُ المُضطَرَّ اِذا دَعاهُ وَ یَکشِفُ السُّوءَ \n\nاَللّهُمَّ صَلِّ عَلی فاطِمَة وَ اَبیها وَ بَعْلِها وَ بَنیها وَ سِرِّ الْمُسْتَوْدَعِ فیها بِعَدَدِ ما اَحَاطَ بِهِ عِلْمُکَ\n\nاللهم اشف کل المریض..\n\n#اللّهمَّ_صَل_عَلےمُحَمَّدﷺوآلِ_مُحَمَّدﷺ_وَعَجِل_فرَجَهُم',),
 ('الحمدلله الذی جعلنا من المتمسکین بولایة امیرالمومنین علی بن ابیطالب و الائمة المعصومین علیهم السلام',),
 ('اَمَّن یُّجیبُ المُضطَرَّ اِذا دَعاهُ وَ یَکشِفُ السُّوءَ \n\nاَللّهُمَّ صَلِّ عَلی فاطِمَة وَ اَبیها وَ بَعْلِها وَ بَنیها وَ سِرِّ الْمُسْتَوْدَعِ فیها بِعَدَدِ ما اَحَاطَ بِهِ عِلْمُکَ\n\nاللهم اشف کل المریض..\n\n#اللّهمَّ_صَل_عَلےمُحَمَّدﷺوآلِ_مُحَمَّدﷺ_وَعَجِل_فرَجَهُم',),
 ('الحمدلله الذی جعلنا من المتمسکین بولایة امیرالمومنین علی بن ابیطالب و الائمة المعصومین علیهم السلام',),
 ('اَمَّن یُّجیبُ المُضطَرَّ اِذا دَعاهُ وَ یَکشِفُ السُّوءَ \n\nاَللّهُمَّ صَلِّ عَلی فاطِمَة وَ اَبیها وَ بَعْلِها وَ بَنیها وَ سِرِّ الْمُسْتَوْدَعِ فیها بِعَدَدِ ما اَحَاطَ بِهِ عِلْمُکَ\n\nاللهم اشف کل المریض..\n\n#ال

***
# $\text{Type #3 User}$

In [ ]:
target_user = grouped_rdd.takeSample(False,1,2001)[0]
target_user

('1574096104450412544',
 [('1232300332497002496', '1735364117530112342', 1),
  ('1249146360584114177', '1735407610952593460', 1),
  ('1577715113679020032', '1735408119700717955', 1),
  ('1454479575015165958', '1735368209698541690', 1),
  ('226928166', '1734521930948972600', 1),
  ('1049384664', '1734603377471062075', 1)])

In [ ]:
target_user_tweets = parsed_rdd.flatMap(lambda x: target_user_tweets_parser(target_user,x))
target_user_tweets.collect()

[('او او او \nصدای مردم ایران علیه دیکتاتور مثل همیشه جهانی شد و حتی به ارتش اسرائیل هم رسید💥\n\n#صادق_بوقی https://t.co/lFZN0C2hG3',
  'generated'),
 ('این چیزها رو نکشید بچه\u200cها 😂 https://t.co/nMb1Q5j61c', 'generated'),
 ("ملت اسپانیا می\u200cتواند بهترین الگوی میهن\u200cپرستان ایرانی در راه خیزش ملی باشد. اسپانیایی\u200cها برای احیاء هویت ملی از اسلام اموی، فاشیسم، جمهوری\u200c، تجزیه\u200cطلب\u200cها و در نهایت از کمونیست\u200cهای چپ\u200cگرا عبور کردند. ایرانیان نیز این توان را دارند. در این مسیر همه چیز به 'سوگند قاهره' برمی\u200cگردد. https://t.co/2AiM1hQWiC",
  'generated'),
 ('13 سال حبس برای #مهسا_یزدانی عزیز مادر جاویدنام #محمدجواد_زاهدی به جرم دادخواهی . \n\nنکته اصلی اینه که با بایکوت کامل مازندران توسط شبکه های خبری و حقوق بشری،دادگاه انقلاب ساری طی یک سال گذشته سنگین ترین احکام رو به راحتی وعدم ترس از افکارعمومی صادر کرد \nهیچوقت از مازندران نگفتید https://t.co/W5NOpA3K9V',
  'generated'),
 ('🔴لشکرکشی به #اسلو نه تنها مُوَلد محبوبیت (اصلاحطلبان)نگردید،بلکه جامعه ی آز

In [ ]:
similars_rdd = grouped_rdd.flatMap(lambda x:pearson_coeff(target_user,x))
similars_rdd.count()

296851

In [ ]:
sorted_similars_rdd = similars_rdd.sortByKey()
sorted_similars_rdd.count()

296851

In [ ]:
similar_users = sorted_similars_rdd.top(20)
similar_users

[(1.3416407864998738, '945011311364198400'),
 (1.3416407864998738, '874899555892047873'),
 (1.3416407864998738, '635905500'),
 (1.3416407864998738, '2862756077'),
 (1.3416407864998738, '2290250094'),
 (1.3416407864998738, '1691135810152681493'),
 (1.3416407864998738, '1685283609635377152'),
 (1.3416407864998738, '1661997428285874177'),
 (1.3416407864998738, '1650891665077334017'),
 (1.3416407864998738, '1643665017366097920'),
 (1.3416407864998738, '1638852367868207106'),
 (1.3416407864998738, '1638667917972602880'),
 (1.3416407864998738, '1637515179394056193'),
 (1.3416407864998738, '1594224521950674946'),
 (1.3416407864998738, '1593036005455511552'),
 (1.3416407864998738, '1593027443861987328'),
 (1.3416407864998738, '1587417286842908672'),
 (1.3416407864998738, '1580960286978236416'),
 (1.3416407864998738, '1580447698200805378'),
 (1.3416407864998738, '1578814191674314762')]

In [ ]:
recomm_tweets_ids = grouped_rdd.flatMap(lambda x: similar_tweets_score(similar_users,x)).collect()

In [ ]:
recomm_tweets_ids.sort(key=lambda a: a[0],reverse = True)

In [ ]:
recommend_tweets_ids = recomm_tweets_ids[0:20]
recommend_tweets_ids

[(4.024922359499621, '1734638622920298899'),
 (4.024922359499621, '1736829679644868705'),
 (4.024922359499621, '1735674240941035753'),
 (4.024922359499621, '1735703356704727448'),
 (4.024922359499621, '1734303030772683235'),
 (4.024922359499621, '1735743796418150653'),
 (4.024922359499621, '1735431247524999614'),
 (4.024922359499621, '1736551527198658808'),
 (4.024922359499621, '1735489509817532581'),
 (4.024922359499621, '1735490187638034513'),
 (4.024922359499621, '1735491111907393642'),
 (4.024922359499621, '1735492658921923071'),
 (4.024922359499621, '1735493682059452843'),
 (4.024922359499621, '1734512533652857019'),
 (4.024922359499621, '1734908880897929434'),
 (4.024922359499621, '1734665362262671482'),
 (4.024922359499621, '1734665362262671482'),
 (4.024922359499621, '1735713588088607109'),
 (4.024922359499621, '1736168545619231053'),
 (4.024922359499621, '1736169799418331389')]

In [ ]:
recommended_tweets = parsed_rdd.flatMap(lambda x: recommended_tweets_parser(recommend_tweets_ids,x))
recommended_tweets.collect()

[('جالبه عکسایی که ترند شده با عکس های شاهنشاه کون دو قشر رو بشدت سوزونده 😂\nاول عرزشیا ک خب تکلیفشون مشخصه\nو قشر دوم چپولای بی وطن\nیعنی هموطن بدون ک اینا عقایدشون به عرزشیا خیلی نزدیکتر از اون چیزیه ک فکرشو کنی \nپس چشاتو باز کن و بعد تصمیم بگیر',),
 ('ماجرای نوبل از دید من:\n۱-بعد\u200c جدی شدن داستان مهسا اصلاح طلبا به فکر یه خمینی زنانه افتادن و بهترین گزینه هم مسیح بود و خب شروع کردن بولد کردنش و شناسوندن مسیح به مردم و بعدم با اوردنش کنار شاهزاده تقریبا به چیزی ک میخواستن رسیدن\nاما بعد این ک جو اروم شد و آب از آسیاب افتاد',),
 ('۲-یمدت همه چی ساکن شد و بعد دوباره اومدن مسیح بولد کنن و چی بهتر از جایزه نوبل که یهو کارما زد به کمر مسیح و رفیقاش و دستش رو شد و همونجاها بود ک امریکا گفتش دیگه نیازی به حمایت امنیتی ما نداری و خب اینجا دست اقایون موند تو حنا ک\u200c چه کنیم چه نکنیم به فکر نرگس محمدی افتادن',),
 ('۳-اما خب نرگس محمدی رو کی میشناسه ؟ هیشکی!\nچطوری بشناسونیم؟کی بهتر از گلشیفته! \nاما خود گلشیفته هم از یاد رفته ! \nچ کنیم ؟\nکمتر از ی ماه قبل از نوبل ، سینا اومد یک ساع

***
# $\text{Type #4 User}$

In [ ]:
target_user = grouped_rdd.takeSample(False,1,777)[0]
target_user

('4702453171',
 [('4702453171', '1734001719539810691', 3),
  ('4702453171', '1734471154646049090', 3),
  ('4702453171', '1734471452450005121', 3),
  ('1653467502972682270', '1734079495177461916', 2),
  ('25710094', '1734490763394798040', 2),
  ('60171166', '1736513161631588535', 2),
  ('17532913', '1734073641908981882', 2),
  ('4702453171', '1733813004280459569', 2),
  ('4702453171', '1734086631366311958', 3),
  ('4702453171', '1734086631366311958', 2),
  ('17532913', '1734087759646871761', 2),
  ('4702453171', '1734096096526291087', 3),
  ('4702453171', '1734099408688488815', 3),
  ('4702453171', '1734099878626705917', 3),
  ('4702453171', '1734100866527817896', 3),
  ('4702453171', '1735916939380212209', 3),
  ('4702453171', '1734103110279102546', 3),
  ('4702453171', '1726253167547924649', 2),
  ('15090494', '1734104725874713032', 2),
  ('360461731', '1734106135055958309', 2),
  ('1271067471760568322', '1734105477301117383', 2),
  ('4702453171', '1734108334402228647', 3),
  ('767126

In [ ]:
target_user_tweets = parsed_rdd.flatMap(lambda x: target_user_tweets_parser(target_user,x))
target_user_tweets.collect()

[('شما فقط قیمت یک اپ مثل #Blinkist رو برای استور ترکیه ببینید.\nمفته واقعا\nحالت عادی ١٠٠ دلاره\nولی من با ٣٠٠ لیر میتونم بخرمش :)) https://t.co/snE0lCFuKD',
  'generated'),
 ('آقا چرا بسته های اینترنتی این کفار اینقدر خوبن آخه.\n\nزده ۲۵ گیگ اینترنت با ۳ ماه یوتیوب پرمیوم و دیتای نامحدود برای واتساپ, یوتیوب, اسپاتیفای. همه اینا برای ۳ ماه اول ۳۷۰ لیر :/\n\nچرا تو ایران همچین چیزایی نداریم 😑\n#Vodafone https://t.co/xoSKCGtk1k',
  'generated'),
 ('چوشی اندرویدم که واقعا وضعیت وخیمی داره رو بعدها میدم تعمیر ولی تا اون موقع به عنوان روتر دارم استفاده میکنم ازش و خیلی خوب شده😁\n\nقدم بعدی حتما خرید یک روتر واقعیه',
  'generated'),
 ('من درک نمیکنم این قضیه رو\n\nگوگل بزرگه\nاپل هم بزرگه\nکلی هم پایبند به قوانین مالی آمریکا هستند.\n\nولی چرا سیستم پرداخت اپل اینقدررر بی دردسر تر از گوگله؟\nبرای گوگل باید کلی سند و مدرک و آیپی رزیدنشال جور کنی، ولی اپل میگه فقط پولو بده بهم و یه روش پرداخت تو اون کشور داشته باش و تمام.\n\nهمین موردم باعث میشه بازم بخوام این گوشیو نگه دارم.',
  'generated'),

In [ ]:
similars_rdd = grouped_rdd.flatMap(lambda x:pearson_coeff(target_user,x))
similars_rdd.count()

296851

In [ ]:
sorted_similars_rdd = similars_rdd.sortByKey()
sorted_similars_rdd.count()

296851

In [ ]:
similar_users = sorted_similars_rdd.top(20)
similar_users

[(1.386750490563073, '855095592'),
 (1.386750490563073, '2564151662'),
 (1.386750490563073, '1572479734335815684'),
 (1.386750490563073, '1206642366830002179'),
 (1.386750490563073, '1152293110212263937'),
 (1.386750490563073, '1151820549653446656'),
 (1.386750490563073, '1141720339'),
 (1.3416407864998738, '941203650818560000'),
 (1.3416407864998738, '927327889'),
 (1.3416407864998738, '880550727680184320'),
 (1.3416407864998738, '860039158370316288'),
 (1.3416407864998738, '818518646968422400'),
 (1.3416407864998738, '801430424'),
 (1.3416407864998738, '75530244'),
 (1.3416407864998738, '720019748780699649'),
 (1.3416407864998738, '712037416345989123'),
 (1.3416407864998738, '36924087'),
 (1.3416407864998738, '36024428'),
 (1.3416407864998738, '30186494'),
 (1.3416407864998738, '2421831930')]

In [ ]:
recomm_tweets_ids = grouped_rdd.flatMap(lambda x: similar_tweets_score(similar_users,x)).collect()

In [ ]:
recomm_tweets_ids.sort(key=lambda a: a[0],reverse = True)

In [ ]:
recommend_tweets_ids = recomm_tweets_ids[0:20]
recommend_tweets_ids

[(4.160251471689219, '1736313883789312092'),
 (4.160251471689219, '1734302996685582511'),
 (4.160251471689219, '1736543598210908218'),
 (4.160251471689219, '1736191078921826671'),
 (4.160251471689219, '1736194209722352050'),
 (4.160251471689219, '1736196909767745970'),
 (4.160251471689219, '1735911029438083239'),
 (4.160251471689219, '1734100975072207178'),
 (4.160251471689219, '1736660798875881627'),
 (4.160251471689219, '1734851686357782537'),
 (4.160251471689219, '1734118177137717730'),
 (4.160251471689219, '1735933230006558790'),
 (4.160251471689219, '1734597062904140077'),
 (4.160251471689219, '1734599210136154571'),
 (4.160251471689219, '1736693297865806316'),
 (4.160251471689219, '1736693537289228516'),
 (4.160251471689219, '1735961908656689410'),
 (4.160251471689219, '1736321041838879073'),
 (4.160251471689219, '1734158029661077888'),
 (4.160251471689219, '1736336769023385988')]

In [ ]:
recommended_tweets = parsed_rdd.flatMap(lambda x: recommended_tweets_parser(recommend_tweets_ids,x))
recommended_tweets.collect()

[('آخیشششش\nفی شبکه بیتکوین یه نفسی کشید :))',),
 ('توکن $SYMM خیلی چیز جالبیه\nهمزمان و بصورت دوگانه روی هردو پروتکل BRC20 و CBRC20 اینسکرایب میشه.\n\nاولین هست در نوع خودش!',),
 ('بخدا این انصاف نیست تایمی که ما باید بخوابیم، آمریکاییا بیدارن همه چیو پامپ میکنن\n\nخب من الان ساعت ۵ صبح با چه حالی بلند بشم اتر بریزم تو ولت؟😑',),
 ('چنان حجم پولی تزریق شده توی اکوسیستم بیتکوین که باور نمیکنید!!\nعجیب و غریب!\n\nبعد ما میریم تسک لینیا میزنیم:))',),
 ('متاسفانه تراکنشهای خودم روی $UTXO تایید نشد و invalid بودن\nولی اگر مینت کردید الان براحتی میتونید ۱۸۰۰ دلار بفروشید :))',),
 ('چه عجب بالاخره یه کندل قرمز هم دیدیم  :)\n\nولکنش به میدلاین بود (قبلا چارتشو تو کانال گذاشته بودم)\n\nببینیم کف، قیمت رو حمایت میکنه یا باید بریم پایینتر…\n\n#bitcoin https://t.co/97we1i01xC',),
 ('اینسکریپشن روی هر بلاکچینی بجز بیتکوین،\nارزش خرید و یا نگهداری نداره!!',),
 ('حواستون به #SRC721 باشه\nپروتکل توکنهای NFT روی SRC20\n\nخبرهای خوبی براش در راهه…',),
 ('دامیننس تتر جای خیلی حساسی هست!\n#USDT.D https://

***
# $\text{Type #5 User}$

In [ ]:
target_user = grouped_rdd.takeSample(False,1,91)[0]
target_user

('1693973461306109952',
 [('1574904470835798041', '1734442494039826873', 2),
  ('1574904470835798041', '1734883099647914204', 2),
  ('1574904470835798041', '1735771172472492519', 2),
  ('1693973461306109952', '1736853602969280923', 3),
  ('1693973461306109952', '1736855151917334935', 3),
  ('1693973461306109952', '1735069630308266029', 3),
  ('1693973461306109952', '1735401606160785610', 3),
  ('1693973461306109952', '1735071498032058504', 3),
  ('1693973461306109952', '1735071498032058504', 2),
  ('1574904470835798041', '1735403820757426674', 2)])

In [ ]:
target_user_tweets = parsed_rdd.flatMap(lambda x: target_user_tweets_parser(target_user,x))
target_user_tweets.collect()

[('«تو » دیگر \nحتی «تو» نیستی\n«شما» شده اید\nو این دنیای مودب جای «ما» نیست\nما دیوانه شده ها \n#افشین_خماند',
  'generated'),
 ('تمام سیاره ها را گشتم تا به جایی رسیدم که خالی از هر کسی بود و آنگاه دلم آرام گریست...\nاز آن روز به بعد شدم باغبان باغچه ی کوچکم با یک گل...\nگلم قرمز ، بنفش و اکنون آبی را در خود دارد...',
  'generated'),
 ('امروز خواستم از عشق بگم، دیدم عشقی نیست! خواستم از عاشقونه ها حرف بزنم دیدم چیزی نیست، یادم اومد از دل تنگ بنویسم که فهمیدم...!!\nنه! هیچی، چیزی نفهمیدم، فقط خوابیدم. همین...',
  'generated'),
 ('اگه دستت بی وزن شد، حرکتش دادی و نتونستی لمسش کنی، باختی\nاگه چشات صدات کرد، سرتو بلند کردی ندیدیش، باختی\nاگه گوشات هوشیار شد، تیزش کردی و صداش رو نشنیدی، باختی\nاگه دلت تنگ شد، موبایلت رو ورداشتی، زنگش زدی، جوابت رو نداد، باختی\nو امشب من بازنده ترینم...',
  'generated'),
 ('یه روز یکی از آدمای اطرافم که دنیاش خیلی رنگیه بهم گفت: چقدر دوسم داری!؟\nخیلی زود جواب دادم دوس داشتن که اندازه نداره رفیق...\nگفت یعنی چی!؟ گفتم دوس داشتن یا هست یا نیست، اندازه ندار

In [ ]:
similars_rdd = grouped_rdd.flatMap(lambda x:pearson_coeff(target_user,x))
similars_rdd.count()

296851

In [ ]:
sorted_similars_rdd = similars_rdd.sortByKey()
sorted_similars_rdd.count()

296851

In [ ]:
similar_users = sorted_similars_rdd.top(20)
similar_users

[(1.0000000000000002, '1693973461306109952'),
 (1.0, '1574904470835798041'),
 (0, '999996903185100800'),
 (0, '99999538'),
 (0, '999995055090929664'),
 (0, '999988080819015680'),
 (0, '999970696435523584'),
 (0, '999964377922134016'),
 (0, '999963219425931266'),
 (0, '999959604556697602'),
 (0, '999956594686349317'),
 (0, '999953030060675072'),
 (0, '999952212783783939'),
 (0, '999951354620727296'),
 (0, '999951276'),
 (0, '999928626987569152'),
 (0, '999925604957937665'),
 (0, '999924334402531328'),
 (0, '999922321816080386'),
 (0, '999918484485128195')]

In [ ]:
recomm_tweets_ids = grouped_rdd.flatMap(lambda x: similar_tweets_score(similar_users,x)).collect()

In [ ]:
recomm_tweets_ids.sort(key=lambda a: a[0],reverse = True)

In [ ]:
recommend_tweets_ids = recomm_tweets_ids[0:20]
recommend_tweets_ids

[(3.000000000000001, '1736853602969280923'),
 (3.000000000000001, '1736855151917334935'),
 (3.000000000000001, '1735069630308266029'),
 (3.000000000000001, '1735401606160785610'),
 (3.000000000000001, '1735071498032058504'),
 (3.0, '1734042842803110258'),
 (3.0, '1736261559083491431'),
 (3.0, '1735326293720228136'),
 (3.0, '1736124195354890571'),
 (3.0, '1735771172472492519'),
 (3.0, '1735406823128125835'),
 (3.0, '1735410144727478743'),
 (3.0, '1735422143100244168'),
 (2.0000000000000004, '1734442494039826873'),
 (2.0000000000000004, '1734883099647914204'),
 (2.0000000000000004, '1735771172472492519'),
 (2.0000000000000004, '1735071498032058504'),
 (2.0000000000000004, '1735403820757426674'),
 (0, '1735767625915859133'),
 (0, '1735872482198860152')]

In [ ]:
recommended_tweets = parsed_rdd.flatMap(lambda x: recommended_tweets_parser(recommend_tweets_ids,x))
recommended_tweets.collect()

[('تو\nآنجایی\nو \n"آنجا" \nنمی داند \nکه\nچقدر\nخوشبخت\nاست!',),
 ('این شریفی\u200cمقدم که مجری محبوب آخونده و تو همه شبکه\u200cها هست(منم بلاک کرده) امروز علنا گوشی آیفون ۱۳پرومکس تو برنامه زنده گرفته دستش\nاینا دیگه تعارف رو کنار گذاشتن و به ریش ملت میخندن',),
 ('بخیر ترین صبح قطعا اونی که \nکنار کسی که دوسش داری شروع میشه…',),
 ('آنجا که تو نیستی، کجا خواهد بود…',),
 ('«تو » دیگر \nحتی «تو» نیستی\n«شما» شده اید\nو این دنیای مودب جای «ما» نیست\nما دیوانه شده ها \n#افشین_خماند',),
 ('تمام سیاره ها را گشتم تا به جایی رسیدم که خالی از هر کسی بود و آنگاه دلم آرام گریست...\nاز آن روز به بعد شدم باغبان باغچه ی کوچکم با یک گل...\nگلم قرمز ، بنفش و اکنون آبی را در خود دارد...',),
 ('گمان میکنی پایان است ،\nو خدا همه چیز را درست میکند…',),
 ('امروز خواستم از عشق بگم، دیدم عشقی نیست! خواستم از عاشقونه ها حرف بزنم دیدم چیزی نیست، یادم اومد از دل تنگ بنویسم که فهمیدم...!!\nنه! هیچی، چیزی نفهمیدم، فقط خوابیدم. همین...',),
 ('اگه دستت بی وزن شد، حرکتش دادی و نتونستی لمسش کنی، باختی\nاگه چشات صدات